# Autoencoder-Based Anomaly Detection for CMS Open Payments

**Project:** AAI-540 Machine Learning Operations - Final Team Project  
**Context:** Continuation of notebook 03 - Feature Engineering & Model Preparation  
**Objective:** Train an Autoencoder model to detect anomalous payment patterns using reconstruction error as anomaly score

---

## Table of Contents
1. [Setup & Data Loading](#setup)
2. [Load Feature Store Data](#loading)
3. [Data Preparation & Normalization](#preparation)
4. [Autoencoder Architecture Design](#architecture)
5. [Model Training with Early Stopping](#training)
6. [Performance Evaluation](#evaluation)
7. [Anomaly Score Calculation](#scoring)
8. [Visualizations & Metrics](#visualizations)
9. [Confusion Matrix & ROC Analysis](#confusion)
10. [Summary & Outputs](#summary)

---

## 1. Setup & Data Loading

Load dependencies and restore configuration from notebook 03 (Feature Engineering).

In [5]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import time
import boto3

print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

TensorFlow Version: 2.18.0
GPU Available: False


In [6]:
# Restore configuration and data from notebook 03
print("Restoring configuration from notebook 03 (Feature Engineering)...")

# Check required variables
required_vars = ['cms_payments_feature_group_name', 'record_identifier_feature_name', 
                 'region', 'bucket', 's3_athena_staging']
                 
try:
    %store -r cms_payments_feature_group_name
    %store -r record_identifier_feature_name
    %store -r featurestore_runtime
    %store -r cms_payments_fg
    %store -r df
    %store -r region
    %store -r bucket
    %store -r s3_athena_staging
    %store -r database_name
    %store -r table_name_parquet
    
    # Validate critical variables
    missing = []
    if 'cms_payments_feature_group_name' not in dir():
        missing.append('cms_payments_feature_group_name')
    if 'region' not in dir():
        missing.append('region')
    if 'bucket' not in dir():
        missing.append('bucket')
        
    if missing:
        raise NameError(f"Missing required variables: {missing}")
    
    print(f"Feature Group: {cms_payments_feature_group_name}")
    print(f"Record Identifier: {record_identifier_feature_name}")
    print(f"Region: {region}")
    print(f"S3 Bucket: {bucket}")
    print(f"Athena Staging: {s3_athena_staging}")
    
except NameError as e:
    print(f"\nERROR: {str(e)}")
    print("\nPREREQUISITE: Please run notebook 03 (03_feature_engineering.ipynb) first")
    print("That notebook creates the Feature Store and stores required variables.")
    raise

Restoring configuration from notebook 03 (Feature Engineering)...
no stored variable or alias cms_payments_fg
Feature Group: cms-payments-fg-07-21-49-56
Record Identifier: covered_recipient_profile_id
Region: us-east-1
S3 Bucket: cmsopenpaymentsystemslight
Athena Staging: s3://cmsopenpaymentsystemslight/athena/staging


## 2. Load Feature Store & Production Data from Athena

Query production S3 parquet data from Feature Store offline store using Athena.

In [7]:
import boto3
import sagemaker
import awswrangler as wr
from sagemaker.session import Session

# Initialize AWS clients
region = boto3.Session().region_name
boto_session = boto3.Session(region_name=region)
sagemaker_session = Session(boto_session=boto_session)
sagemaker_client = boto_session.client("sagemaker", region_name=region)

print(f"Region: {region}")
print(f"Athena Staging: {s3_athena_staging}")

Region: us-east-1
Athena Staging: s3://cmsopenpaymentsystemslight/athena/staging


In [8]:
# Load production data from Feature Store offline store
print(f"\nFeature Group: {cms_payments_feature_group_name}")

start_load = time.time()

try:
    # Get Feature Store offline table information
    print("\nAttempting to load from Feature Store offline table...")
    
    # Connect to Feature Group to get offline table name
    from sagemaker.feature_store.feature_group import FeatureGroup
    import sagemaker
    
    # Use stored Feature Group object or recreate it
    if 'cms_payments_fg' in dir() and cms_payments_fg is not None:
        feature_group = cms_payments_fg
        print("  Using stored FeatureGroup object")
    else:
        # Recreate Feature Group connection
        sagemaker_session = sagemaker.Session()
        feature_group = FeatureGroup(
            name=cms_payments_feature_group_name,
            sagemaker_session=sagemaker_session
        )
        print("Recreated FeatureGroup connection")
    
    # Get account ID for offline table name
    sts_client = boto3.client('sts')
    account_id = sts_client.get_caller_identity()['Account']
    
    # Feature Store offline table follows naming convention
    offline_table_name = f"{cms_payments_feature_group_name.replace('-', '_')}_{account_id}_offline"
    
    # Query Feature Store offline data using Athena
    query = f"""
    SELECT * 
    FROM "sagemaker_featurestore"."{offline_table_name}"
    WHERE is_deleted = false
    LIMIT 1000000
    """
    
    print(f"Offline table: {offline_table_name}")
    print(f"Executing Athena query...")
    
    # Query using awswrangler
    df_payments = wr.athena.read_sql_query(
        sql=query,
        database="sagemaker_featurestore",
        s3_output=s3_athena_staging,
        ctas_approach=False
    )
    
    load_time = time.time() - start_load
    
    print(f"\nData loaded from Feature Store in {load_time:.2f} seconds")
    print(f"Shape: {df_payments.shape[0]:,} rows x {df_payments.shape[1]} columns")
    print(f"Memory: {df_payments.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
    print(f"Source: Feature Store offline table (engineered features)")
    
except Exception as e:
    print(f"\nError querying Feature Store: {str(e)[:100]}")

# Display data preview
print("Data Preview:")
print(f"Columns: {df_payments.shape[1]}")
print(f"Data Types: {df_payments.dtypes.value_counts().to_dict()}")
print(f"Sample features: {list(df_payments.columns[:10])}")

# Check for engineered features
engineered_features = ['hist_pay_avg', 'amt_to_avg_ratio', 'is_new_recipient']
present_features = [f for f in engineered_features if f in df_payments.columns]
print(f"Engineered features present: {len(present_features)}/{len(engineered_features)}")

if len(present_features) < len(engineered_features):
    missing = [f for f in engineered_features if f not in df_payments.columns]
    print(f"WARNING: Missing engineered features: {missing}")


display(df_payments.head(3))


Feature Group: cms-payments-fg-07-21-49-56

Attempting to load from Feature Store offline table...
Recreated FeatureGroup connection
Offline table: cms_payments_fg_07_21_49_56_864106638709_offline
Executing Athena query...

Error querying Feature Store: TABLE_NOT_FOUND: line 2:10: Table 'awsdatacatalog.sagemaker_featurestore.cms_payments_fg_07_21_49_56
Data Preview:


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:65                                                                                   │
│                                                                                                  │
│   62                                                                                             │
│   63 # Display data preview                                                                      │
│   64 print("Data Preview:")                                                                      │
│ ❱ 65 print(f"Columns: {df_payments.shape[1]}")                                                   │
│   66 print(f"Data Types: {df_payments.dtypes.value_counts().to_dict()}")                         │
│   67 print(f"Sample features: {list(df_payments.columns[:10])}")                                 │
│   68                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'df_payments' is not defined

## 3. Data Preparation & Normalization

Prepare features for Autoencoder training with appropriate scaling.

In [ ]:
# Select numeric features for anomaly detection
numeric_cols = df_payments.select_dtypes(include=[np.number]).columns.tolist()

# Remove unwanted columns: IDs, timestamps, encoded text fields, and metadata
cols_to_exclude = [
    'EventTime', 'covered_recipient_profile_id', 'index',
    'teaching_hospital_id', 'covered_recipient_npi',
    'covered_recipient_first_name', 'covered_recipient_middle_name',
    'covered_recipient_last_name', 'covered_recipient_name_suffix',
    'recipient_primary_business_street_address_line2',
    'recipient_zip_code', 'recipient_province', 'recipient_postal_code',
    'submitting_applicable_manufacturer_or_applicable_gpo_name',
    'applicable_manufacturer_or_applicable_gpo_making_payment_id',
    'applicable_manufacturer_or_applicable_gpo_making_payment_name'
]

# Filter numeric features - keep only actual numeric measurements
numeric_features = [col for col in numeric_cols 
                   if col not in cols_to_exclude 
                   and not any(x in col.lower() for x in ['_id', '_name', '_address', '_code', '_province', '_postal'])]

print(f"Total numeric features selected: {len(numeric_features)}")
print(f"Sample features: {numeric_features[:15]}")

# Prepare feature matrix - convert to float
X = df_payments[numeric_features].copy().astype(float)

# Handle infinity values
X = X.replace([np.inf, -np.inf], np.nan)

# Check initial missing value percentage per column
missing_pct = (X.isnull().sum() / len(X)) * 100
print(f"\nColumns with >50% missing: {(missing_pct > 50).sum()}")

# Drop columns with more than 50% missing values
cols_to_keep = missing_pct[missing_pct <= 50].index.tolist()
X = X[cols_to_keep]
print(f"Features after dropping high-missing columns: {len(cols_to_keep)}")

# Clip extreme outliers using IQR method for each column
for col in X.columns:
    q1 = X[col].quantile(0.25)
    q3 = X[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 3 * iqr
    upper_bound = q3 + 3 * iqr
    X[col] = X[col].clip(lower=lower_bound, upper=upper_bound)

# Handle remaining missing values with median
X = X.fillna(X.median())

# Final validation
print(f"\nFinal Data shape: {X.shape}")
print(f"Missing values: {X.isnull().sum().sum()}")
print(f"Infinity values: {np.isinf(X.values).sum()}")
print(f"Features range - Min: {X.min().min():.2f}, Max: {X.max().max():.2f}")
print(f"Sample feature names: {X.columns.tolist()[:10]}")

In [ ]:
# Normalize features using MinMaxScaler (0-1 range) for better Autoencoder performance
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

print(f"Scaled data shape: {X_scaled.shape}")
print(f"Scaled range - Min: {X_scaled.min().min():.4f}, Max: {X_scaled.max().max():.4f}")
print(f"Mean: {X_scaled.mean().mean():.4f}, Std: {X_scaled.std().mean():.4f}")

# Split into train and test sets (80-20 split)
# Training set contains mostly normal data; test set may contain anomalies
X_train, X_test = train_test_split(
    X_scaled, 
    test_size=0.2, 
    random_state=42
)

print(f"\nTraining set size: {len(X_train):,}")
print(f"Test set size: {len(X_test):,}")
print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

## 4. Autoencoder Architecture Design

Design a deep Autoencoder with bottleneck layer for feature compression.

In [ ]:
# Autoencoder architecture parameters
input_dim = X_scaled.shape[1]
encoding_dim_1 = max(input_dim // 2, 32)
encoding_dim_2 = max(input_dim // 4, 16)
bottleneck_dim = max(input_dim // 8, 8)

print(f"Input Dimension: {input_dim}")
print(f"Encoding Layer 1: {encoding_dim_1}")
print(f"Encoding Layer 2: {encoding_dim_2}")
print(f"Bottleneck Dimension: {bottleneck_dim}")

# Build Autoencoder model
autoencoder = Sequential([
    # Encoder
    layers.Dense(
        encoding_dim_1, 
        activation='relu', 
        input_shape=(input_dim,),
        name='encoder_input'
    ),
    layers.Dropout(0.2),
    
    layers.Dense(
        encoding_dim_2, 
        activation='relu',
        name='encoder_middle'
    ),
    layers.Dropout(0.2),
    
    layers.Dense(
        bottleneck_dim, 
        activation='relu',
        name='bottleneck'
    ),
    
    # Decoder
    layers.Dense(
        encoding_dim_2, 
        activation='relu',
        name='decoder_middle'
    ),
    layers.Dropout(0.2),
    
    layers.Dense(
        encoding_dim_1, 
        activation='relu',
        name='decoder_layer'
    ),
    layers.Dropout(0.2),
    
    layers.Dense(
        input_dim, 
        activation='sigmoid',
        name='decoder_output'
    )
], name='Autoencoder')

# Display model architecture
autoencoder.summary()

## 5. Model Training with Early Stopping

Train the Autoencoder with optimal configuration and early stopping to prevent overfitting.

In [ ]:
# Compile model with Adam optimizer
optimizer = Adam(learning_rate=0.001)
autoencoder.compile(
    optimizer=optimizer,
    loss='mse',
    metrics=['mae']
)

print("Model compiled successfully")
print(f"Optimizer: Adam (lr=0.001)")
print(f"Loss function: Mean Squared Error (MSE)")
print(f"Metric: Mean Absolute Error (MAE)")

In [ ]:
# Define callbacks for training
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1,
    min_delta=1e-5
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

print("Training configuration configured:")
print("Early Stopping: patience=10, min_delta=1e-5")
print("Reduce LR on Plateau: factor=0.5, patience=5")

In [ ]:
# Train the Autoencoder
print("Starting Autoencoder training...")
print(f"Training samples: {len(X_train):,}")
print(f"Validation samples (20% of train): Test set size {len(X_test):,}")

start_time = time.time()

history = autoencoder.fit(
    X_train, X_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

training_time = time.time() - start_time
print(f"\nTraining completed in {training_time:.2f} seconds")
print(f"Total epochs trained: {len(history.history['loss'])}")

## 6. Performance Evaluation

Evaluate model performance on training and test sets.

In [ ]:
# Evaluate on training set
train_predictions = autoencoder.predict(X_train, verbose=0)
train_mse = np.mean(np.square(X_train - train_predictions), axis=1)

# Evaluate on test set
test_predictions = autoencoder.predict(X_test, verbose=0)
test_mse = np.mean(np.square(X_test - test_predictions), axis=1)

print("Model Evaluation:")
print(f"\nTraining Set:")
print(f"Reconstruction MSE - Mean: {train_mse.mean():.6f}, Std: {train_mse.std():.6f}")
print(f"Reconstruction MSE - Min: {train_mse.min():.6f}, Max: {train_mse.max():.6f}")

print(f"\nTest Set:")
print(f"Reconstruction MSE - Mean: {test_mse.mean():.6f}, Std: {test_mse.std():.6f}")
print(f"Reconstruction MSE - Min: {test_mse.min():.6f}, Max: {test_mse.max():.6f}")

# Model performance metrics
train_final_loss = history.history['loss'][-1]
train_final_mae = history.history['mae'][-1]
val_final_loss = history.history['val_loss'][-1]
val_final_mae = history.history['val_mae'][-1]

print(f"\nFinal Epoch Performance:")
print(f"Training Loss (MSE): {train_final_loss:.6f}, MAE: {train_final_mae:.6f}")
print(f"Validation Loss (MSE): {val_final_loss:.6f}, MAE: {val_final_mae:.6f}")

## 7. Anomaly Score Calculation

Generate anomaly scores and identify outliers using reconstruction error threshold.

In [ ]:
# Combine train and test predictions for comprehensive analysis
all_data = np.vstack([X_train, X_test])
all_predictions = autoencoder.predict(all_data, verbose=0)
all_reconstruction_errors = np.mean(np.square(all_data - all_predictions), axis=1)

# Calculate anomaly threshold (95th percentile of training errors)
threshold_percentile = 95
threshold = np.percentile(train_mse, threshold_percentile)

print(f"Anomaly Detection Configuration:")
print(f"Percentile for threshold: {threshold_percentile}")
print(f"Anomaly threshold: {threshold:.6f}")

# Label anomalies (1 = anomaly, 0 = normal)
anomaly_labels = (all_reconstruction_errors > threshold).astype(int)
anomaly_count = anomaly_labels.sum()
anomaly_percentage = (anomaly_count / len(anomaly_labels)) * 100

print(f"\nAnomaly Detection Results:")
print(f"Total records: {len(anomaly_labels):,}")
print(f"Anomalies detected: {anomaly_count:,} ({anomaly_percentage:.2f}%)")
print(f"Normal records: {len(anomaly_labels) - anomaly_count:,}")

# Anomaly score distribution
print(f"\nReconstruction Error (Anomaly Score) Statistics:")
print(f"Mean: {all_reconstruction_errors.mean():.6f}")
print(f"Median: {np.median(all_reconstruction_errors):.6f}")
print(f"Std Dev: {all_reconstruction_errors.std():.6f}")
print(f"Min: {all_reconstruction_errors.min():.6f}")
print(f"Max: {all_reconstruction_errors.max():.6f}")

## 8. Visualizations & Metrics

Visualize training history, loss distributions, and anomaly scores.

In [ ]:
# Create comprehensive visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Training and Validation Loss
ax1 = axes[0, 0]
ax1.plot(history.history['loss'], label='Training Loss', linewidth=2)
ax1.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=11, fontweight='bold')
ax1.set_ylabel('Loss (MSE)', fontsize=11, fontweight='bold')
ax1.set_title('Model Training History - Loss', fontsize=12, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: Training and Validation MAE
ax2 = axes[0, 1]
ax2.plot(history.history['mae'], label='Training MAE', linewidth=2)
ax2.plot(history.history['val_mae'], label='Validation MAE', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=11, fontweight='bold')
ax2.set_ylabel('MAE', fontsize=11, fontweight='bold')
ax2.set_title('Model Training History - MAE', fontsize=12, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

# Plot 3: Reconstruction Error Distribution
ax3 = axes[1, 0]
ax3.hist(train_mse, bins=50, alpha=0.7, label='Training', color='blue', edgecolor='black')
ax3.hist(test_mse, bins=50, alpha=0.7, label='Test', color='red', edgecolor='black')
ax3.axvline(threshold, color='green', linestyle='--', linewidth=2.5, label=f'Threshold ({threshold:.6f})')
ax3.set_xlabel('Reconstruction Error (MSE)', fontsize=11, fontweight='bold')
ax3.set_ylabel('Frequency', fontsize=11, fontweight='bold')
ax3.set_title('Distribution of Reconstruction Errors', fontsize=12, fontweight='bold')
ax3.legend(fontsize=10)
ax3.grid(True, alpha=0.3)

# Plot 4: Anomaly Score Distribution
ax4 = axes[1, 1]
ax4.hist(all_reconstruction_errors, bins=100, alpha=0.8, color='purple', edgecolor='black')
ax4.axvline(threshold, color='red', linestyle='--', linewidth=2.5, label=f'Anomaly Threshold')
ax4.axvline(all_reconstruction_errors.mean(), color='orange', linestyle=':', linewidth=2.5, label='Mean Score')
ax4.set_xlabel('Anomaly Score', fontsize=11, fontweight='bold')
ax4.set_ylabel('Frequency', fontsize=11, fontweight='bold')
ax4.set_title('Complete Anomaly Score Distribution', fontsize=12, fontweight='bold')
ax4.legend(fontsize=10)
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('autoencoder_training_analysis.png', dpi=150, bbox_inches='tight')
print("Training analysis visualization saved")
plt.show()

In [ ]:
# Create additional visualization: Anomaly score by percentile
fig, ax = plt.subplots(figsize=(12, 6))

sorted_errors = np.sort(all_reconstruction_errors)
percentiles = np.arange(1, 101)

ax.plot(percentiles, np.percentile(all_reconstruction_errors, percentiles), 
        linewidth=2.5, color='darkblue', marker='o', markersize=3)
ax.axhline(y=threshold, color='red', linestyle='--', linewidth=2.5, 
           label=f'95th Percentile Threshold: {threshold:.6f}')
ax.fill_between(percentiles, 0, np.percentile(all_reconstruction_errors, percentiles), 
                alpha=0.2, color='blue')
ax.set_xlabel('Percentile', fontsize=12, fontweight='bold')
ax.set_ylabel('Anomaly Score', fontsize=12, fontweight='bold')
ax.set_title('Anomaly Score Percentile Distribution', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Confusion Matrix & ROC Analysis

Analyze model performance with statistical metrics.

In [ ]:
# Create synthetic labels for evaluation (assuming normal data in train set, potential anomalies in test)
y_train = np.zeros(len(X_train))  # All training data is normal
y_test = np.zeros(len(X_test))    # Test labels (normally would have true labels)

# Combine for analysis
y_true = np.hstack([y_train, y_test])
y_pred = anomaly_labels

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = cm.ravel()

print("Confusion Matrix Analysis:")
print(f"\n{cm}")
print(f"\nTrue Negatives (TN): {tn:,}")
print(f"False Positives (FP): {fp:,}")
print(f"False Negatives (FN): {fn:,}")
print(f"True Positives (TP): {tp:,}")

# Calculate metrics
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
f1 = 2 * (precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0

print(f"\nPerformance Metrics:")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1-Score: {f1:.4f}")

# Try to calculate ROC-AUC with continuous scores
try:
    roc_auc = roc_auc_score(y_true, all_reconstruction_errors)
    print(f"  ROC-AUC Score: {roc_auc:.4f}")
except:
    print("  ROC-AUC: Unable to compute (possibly all samples in one class)")

In [ ]:
# Visualize Confusion Matrix
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion Matrix Heatmap
ax1 = axes[0]
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, ax=ax1,
            xticklabels=['Normal', 'Anomaly'],
            yticklabels=['Normal', 'Anomaly'])
ax1.set_ylabel('True Label', fontsize=11, fontweight='bold')
ax1.set_xlabel('Predicted Label', fontsize=11, fontweight='bold')
ax1.set_title('Confusion Matrix', fontsize=12, fontweight='bold')

# Metrics Comparison
ax2 = axes[1]
metrics = ['Sensitivity', 'Specificity', 'Precision', 'F1-Score']
values = [sensitivity, specificity, precision, f1]
colors_bar = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

bars = ax2.bar(metrics, values, color=colors_bar, edgecolor='black', linewidth=1.5)
ax2.set_ylabel('Score', fontsize=11, fontweight='bold')
ax2.set_title('Performance Metrics', fontsize=12, fontweight='bold')
ax2.set_ylim([0, 1.1])
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, val in zip(bars, values):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# ROC Curve (if enough variation in classes)
try:
    fpr, tpr, thresholds = roc_curve(y_true, all_reconstruction_errors)
    roc_auc = auc(fpr, tpr)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.plot(fpr, tpr, color='darkorange', lw=2.5, label=f'ROC Curve (AUC = {roc_auc:.3f})')
    ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
    ax.set_xlim([-0.01, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate', fontsize=12, fontweight='bold')
    ax.set_ylabel('True Positive Rate', fontsize=12, fontweight='bold')
    ax.set_title('ROC Curve - Autoencoder Anomaly Detection', fontsize=13, fontweight='bold')
    ax.legend(loc="lower right", fontsize=11)
    ax.grid(True, alpha=0.3)    
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"ROC curve generation note: {str(e)[:100]}")

## 10. Summary & Outputs

Save model and anomaly detection results for downstream analysis.

In [ ]:
# Save the trained model
model_path = 'cms_autoencoder_model'
autoencoder.save(model_path)
print(f"Model saved to: {model_path}")

# Save scaler for future inference
import pickle
scaler_path = 'feature_scaler.pkl'
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f"Feature scaler saved to: {scaler_path}")

# Create results summary
results_summary = pd.DataFrame({
    'Metric': ['Total Records', 'Anomalies Detected', 'Anomaly Percentage', 
               'Anomaly Threshold', 'Mean Anomaly Score', 'Training Time (sec)',
               'Epochs Trained', 'Final Training Loss', 'Final Validation Loss'],
    'Value': [len(anomaly_labels), anomaly_count, f'{anomaly_percentage:.2f}%',
              f'{threshold:.6f}', f'{all_reconstruction_errors.mean():.6f}',
              f'{training_time:.2f}', len(history.history['loss']),
              f'{train_final_loss:.6f}', f'{val_final_loss:.6f}']
})

print("\nExecution Summary:")
print(results_summary.to_string(index=False))